https://github.com/microsoft/autogen/blob/main/README.md

# Auto Generated Agent Chat: Solving Tasks Requiring Web Info

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform tasks which require acquiring info from the web:
* discuss a paper based on its URL.
* discuss about the stock market.

Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "TERMINATE", the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`, and prompt the user for feedback when the task is finished. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install pyautogen and docker:
```bash
pip install pyautogen docker
```

In [1]:
%pip install "pyautogen>=0.2.3" docker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [3]:
# only for Google Colab; please comment Kaggle part in this case
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# only for Kaggle; please comment Google Colab part in this case
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#openai_api_key = user_secrets.get_secret("OPENAI_API_KEY")

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [4]:
import autogen
from google.colab import userdata

config_list = [
    {
        'model': 'gpt-4',
        'api_key': openai_api_key,
    }
]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for human feedback when it receives a "TERMINATE" signal from the assistant agent.

In [5]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

## Example Task: Paper Talk from URL

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [6]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
Who should read this paper: https://arxiv.org/abs/2308.08155
""",
)

user_proxy (to assistant):


Who should read this paper: https://arxiv.org/abs/2308.08155


--------------------------------------------------------------------------------
assistant (to user_proxy):

To determine who should read the paper, we need to first understand the content and context of the paper. We can do this by fetching the abstract of the paper from the provided URL and analyzing it. 

Here is a Python script that uses the BeautifulSoup library to scrape the abstract from the webpage. 

```python
# filename: fetch_abstract.py

import requests
from bs4 import BeautifulSoup

def fetch_abstract(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    abstract = soup.find('blockquote', attrs={'class': 'abstract mathjax'}).text.strip()
    return abstract

url = "https://arxiv.org/abs/2308.08155"
print(fetch_abstract(url))
```

Please run this script to fetch the abstract of the paper. After that, I will analyze the abstract and suggest 

ChatResult(chat_id=None, chat_history=[{'content': '\nWho should read this paper: https://arxiv.org/abs/2308.08155\n', 'role': 'assistant'}, {'content': 'To determine who should read the paper, we need to first understand the content and context of the paper. We can do this by fetching the abstract of the paper from the provided URL and analyzing it. \n\nHere is a Python script that uses the BeautifulSoup library to scrape the abstract from the webpage. \n\n```python\n# filename: fetch_abstract.py\n\nimport requests\nfrom bs4 import BeautifulSoup\n\ndef fetch_abstract(url):\n    response = requests.get(url)\n    soup = BeautifulSoup(response.text, \'html.parser\')\n    abstract = soup.find(\'blockquote\', attrs={\'class\': \'abstract mathjax\'}).text.strip()\n    return abstract\n\nurl = "https://arxiv.org/abs/2308.08155"\nprint(fetch_abstract(url))\n```\n\nPlease run this script to fetch the abstract of the paper. After that, I will analyze the abstract and suggest who should read the

## Example Task: Chat about Stock Market

In [7]:
user_proxy.initiate_chat(
    assistant,
    message="""Show me the YTD gain of 10 largest technology companies as of today.""",
)

user_proxy (to assistant):

Show me the YTD gain of 10 largest technology companies as of today.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To get the YTD gain of the 10 largest technology companies, we need to do the following steps:

1. Identify the 10 largest technology companies. We can use the market capitalization to determine the size of the companies. We can get this information from a financial data provider. For this task, we will use Yahoo Finance as it provides free access to this data.

2. Get the YTD gain for each of these companies. The YTD gain is the percentage change in the price of the stock from the first trading day of the year to the current date. We can also get this information from Yahoo Finance.

We can use Python to automate these steps. Python has several libraries that we can use to scrape data from the web and to handle and analyze the data. For this task, we will use the `requests` and `be

ChatResult(chat_id=None, chat_history=[{'content': 'Show me the YTD gain of 10 largest technology companies as of today.', 'role': 'assistant'}, {'content': 'To get the YTD gain of the 10 largest technology companies, we need to do the following steps:\n\n1. Identify the 10 largest technology companies. We can use the market capitalization to determine the size of the companies. We can get this information from a financial data provider. For this task, we will use Yahoo Finance as it provides free access to this data.\n\n2. Get the YTD gain for each of these companies. The YTD gain is the percentage change in the price of the stock from the first trading day of the year to the current date. We can also get this information from Yahoo Finance.\n\nWe can use Python to automate these steps. Python has several libraries that we can use to scrape data from the web and to handle and analyze the data. For this task, we will use the `requests` and `beautifulsoup4` libraries to scrape the data,